In [1]:
import mdtraj as md
from pathlib import Path
import torch
import numpy as np
import sys
sys.path.append('../')
from molgen.models import DDPM

In [2]:
pdb_fname = '/project/andrewferguson/Kirill/CMSC-35450/data_mdshare/alanine-dipeptide-nowater.pdb'
trj_fnames = [str(i) for i in Path('/project/andrewferguson/Kirill/CMSC-35450/data_mdshare').glob('alanine-dipeptide-*-250ns-nowater.xtc')]
trjs  = [md.load(t, top=pdb_fname).center_coordinates() for t in trj_fnames]

In [3]:
xyz = list()
phi_psi = list()
for trj in trjs:
    
    t_backbone = trj.center_coordinates()
    
    n = trj.xyz.shape[0]
    
    _, phi = md.compute_phi(trj)
    _, psi = md.compute_psi(trj)
    
    xyz.append(torch.tensor(t_backbone.xyz.reshape(n, -1)).float())
    phi_psi.append(torch.tensor(np.concatenate((phi, psi), -1)).float())
    
xyz[0].shape, phi_psi[0].shape

(torch.Size([250000, 66]), torch.Size([250000, 2]))

In [4]:
model = DDPM(xyz[0].shape[1], phi_psi[0].shape[1])

In [5]:
model.fit(xyz, phi_psi, max_epochs=25)

/project/andrewferguson/Kirill/class_project_env/lib/python3.7/site-packages/lightning_lite/plugins/environments/slurm.py:170: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /project/andrewferguson/Kirill/class_project_env/lib ...
  category=PossibleUserWarning,
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type              | Params
--------------------------------------------------------
0 | model             | GaussianDiffusion | 3.0 M 
1 | ema_model         | GaussianDiffusion | 3.0 M 
2 | _feature_scaler   | MinMaxScaler      | 0     
3 | _condition_scaler | MinMaxScaler      | 0     
--------------------------------------------------------
6.0 M     Trainable 

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=25` reached.


DDPM(
  (model): GaussianDiffusion(
    (denoise_fn): Unet(
      (time_pos_emb): SinusoidalPosEmb()
      (mlp): Sequential(
        (0): Linear(in_features=32, out_features=128, bias=True)
        (1): Mish()
        (2): Linear(in_features=128, out_features=32, bias=True)
      )
      (downs): ModuleList(
        (0): ModuleList(
          (0): ResnetBlock(
            (mlp): Sequential(
              (0): Mish()
              (1): Linear(in_features=32, out_features=32, bias=True)
            )
            (block1): Block(
              (block): Sequential(
                (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(1,))
                (1): GroupNorm(8, 32, eps=1e-05, affine=True)
                (2): Mish()
              )
            )
            (block2): Block(
              (block): Sequential(
                (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,))
                (1): GroupNorm(8, 32, eps=1e-05, affine=True)
                (2): Mis

In [6]:
import nglview as nv
trj_backbones = md.join(trjs)
v = nv.show_mdtraj(trj_backbones)
v

NGLWidget(max_frame=749999)

In [7]:
#xyz = model.generate(torch.cat(phi_psi))
xyz = model.generate(torch.cat(phi_psi)[::1000])

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

In [8]:
xyz = xyz.reshape(xyz.size(0), -1, 3)
fake_trj = md.Trajectory(xyz = xyz.cpu().numpy(), topology = trj_backbones.top)
fake_trj

<mdtraj.Trajectory with 7500 frames, 22 atoms, 3 residues, without unitcells at 0x7f58e320bf28>

In [9]:
v = nv.show_mdtraj(fake_trj)
v

NGLWidget(max_frame=7499)